<a href="https://colab.research.google.com/github/SumitJha406/Machine_learning_projects/blob/main/Ensemble%20learning%20using%20stacknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/h2oai/pystacknet



Cloning into 'pystacknet'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), done.


In [ ]:
%cd '/content/pystacknet'



/content/pystacknet


In [ ]:
!python /content/pystacknet/setup.py install

running install
running bdist_egg
running egg_info
creating pystacknet.egg-info
writing pystacknet.egg-info/PKG-INFO
writing dependency_links to pystacknet.egg-info/dependency_links.txt
writing requirements to pystacknet.egg-info/requires.txt
writing top-level names to pystacknet.egg-info/top_level.txt
writing manifest file 'pystacknet.egg-info/SOURCES.txt'
adding license file 'LICENSE.txt'
writing manifest file 'pystacknet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pystacknet
copying pystacknet/__init__.py -> build/lib/pystacknet
copying pystacknet/metrics.py -> build/lib/pystacknet
copying pystacknet/pystacknet.py -> build/lib/pystacknet
creating build/lib/pystacknet/test
copying pystacknet/test/__init__.py -> build/lib/pystacknet/test
copying pystacknet/test/test_pystacknet.py -> build/lib/pystacknet/test
copying pystacknet/test/test_amazon.py -> build/lib/pys

In [ ]:
import sys
sys.path


['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
sys.path.append('/content/pystacknet')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pystacknet.pystacknet import StackNetClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%cd ../

/content


In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle-sep_tabular_competition/data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle-sep_tabular_competition/data/test.csv')


In [ ]:
def int_downcast(df):
    int_cols = df.select_dtypes(include=['int64'])

    for col in int_cols.columns:
        print(col, 'min:',df[col].min(),'; max:',df[col].max())
        df[col] = pd.to_numeric(df[col], downcast ='integer')
    return df

int_downcast(train_data)
# test_data['claim'].dtype

id min: 0 ; max: 957918
claim min: 0 ; max: 1


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.108590,0.004314,-37.566,0.017364,0.289150,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.3000,8.6331,4.060700,26.8670,-1.18000,10961.00,1.5397,135.3200,-1.496500,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.750500,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.00,...,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.39800,-2.2012,6871.00,3.8862,-0.005580,5252.100,166.6900,1.607400,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.4710,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.110930,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.10,1.01770,1
1,1,0.100900,0.299610,11822.000,0.276500,0.459700,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.2840,6.1872,1.041900,4.6404,31.87700,123620.00,1.3951,125.8100,1.198900,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.503300,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.50,...,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.05100,205.6900,4295.30,13.3880,0.468430,754.610,83.2330,1.189000,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.7200,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.976730,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.00,0.28359,0
2,2,0.178030,-0.006980,907.270,0.272140,0.459480,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.8730,7.5463,1.996700,1.9526,817.76000,-2948.70,2.0054,1.6826,1.196800,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.131800,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.40,...,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.37310,391.3700,2965.30,NaN,0.494590,43.524,138.5200,1.107900,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.4860,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.201020,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.20,0.40690,1
3,3,0.152360,0.007259,780.100,0.025179,0.519470,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.8400,7.8490,1.037900,8.0030,12.34900,-195.28,2.5598,92.1420,0.637890,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.989410,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.50,...,0.372830,1.5606,1.835400e+10,-3.4298,6485700.0,2120.000,3.081200e+10,34.05600,157.4300,3724.50,8.4211,0.407780,2971.200,204.7000,-0.979980,9.94050,12011.000,0.99898,5.063400e+13,1.2261,0.250200,0.72974,373690.0,194.6500,120.930,0.260710,0.234240,-0.002794,1.442300e+09,-0.011820,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.10,0.51486,1
4,4,0.116230,0.502900,-109.150,0.297910,0.344900,-0.40932,2538.90,65332.0,1.907200e+15,144.120,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.9000,7.2783,3.975700,NaN,NaN,29520.00,3.4225,96.7250,0.797250,215.570,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.974130,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.10,...,0.085690,1.5846,3.825200e+10,130.7000,102100.0,1951.800,1.142800e+10,58.56600,176.8300,1279.00,4.9662,0.479120,-70.278,10.8870,1.143400,6.19120,197.470,NaN,1.574800e+13,1.0083,0.339530,13.48600,201300.0,38.8420,324.000,0.238250,0.141550,0.002208,5.830700e+09,0.927390,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
def float_downcast(df):
  float_cols = df.select_dtypes(include = ['float64'])
  for col in float_cols.columns:
    df[col] = pd.to_numeric(df[col],downcast = 'float')
  return df

float_downcast(train_data)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 120 entries, id to claim
dtypes: float32(118), int32(1), int8(1)
memory usage: 435.8 MB


In [ ]:
features = [x for x in train_data.columns.values if x[0]=="f"]
train_data['n_missing'] = train_data[features].isna().sum(axis=1)
test_data['n_missing'] = test_data[features].isna().sum(axis=1)

In [ ]:
train_data['kurt'] = train_data[features].kurtosis(axis=1)
test_data['kurt'] = test_data[features].kurtosis(axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
imputer =SimpleImputer(np.nan , strategy = 'mean')

In [ ]:
train_data

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim,n_missing,kurt
0,0,0.108590,0.004314,-37.566002,0.017364,0.289150,-10.25100,135.119995,168900.0,3.992400e+14,86.488998,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300049,8.6331,4.060700,26.867001,-1.180000,10961.000000,1.5397,135.320007,-1.496500,440.079987,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.750500,18509.000000,146820.0,-0.000276,1.090600e+16,1705.400024,414.290009,3.5392,1888.000000,...,1.231800e+11,275.920013,5308500.0,1704.000000,5.022400e+10,53.397999,-2.201200,6871.000000,3.8862,-0.005580,5252.100098,166.690002,1.607400,0.665340,7768.899902,0.99662,1.125700e+11,2.2432,0.934160,0.650560,94569.000000,21.471001,8214.099609,0.288010,0.097826,0.001071,1.412400e+09,0.110930,-12.228000,1.7482,1.90960,-7.11570,4378.799805,1.2096,8.613400e+14,140.100006,1.01770,1,1,114.055313
1,1,0.100900,0.299610,11822.000000,0.276500,0.459700,-0.83733,1721.900024,119810.0,3.874100e+15,9953.599609,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284000,6.1872,1.041900,4.640400,31.877001,123620.000000,1.3951,125.809998,1.198900,136.449997,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.503300,238000.000000,21440.0,-0.001344,3.079400e+16,229.100006,844.820007,1.4680,4726.500000,...,1.832400e+11,9.651000,32800.0,1480.599976,2.300600e+10,44.050999,205.690002,4295.299805,13.3880,0.468430,754.609985,83.233002,1.189000,29.549999,7343.700195,0.99815,4.877700e+13,1.2708,-0.000969,5.295200,6779.000000,227.720001,34.341999,0.340300,0.143370,0.049276,1.903200e+09,0.976730,-56.757999,4.1684,0.34808,4.14200,913.229980,1.2464,7.575100e+15,1861.000000,0.28359,0,0,99.411774
2,2,0.178030,-0.006980,907.270020,0.272140,0.459480,0.17327,2298.000000,360650.0,1.224500e+13,15827.000000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.872999,7.5463,1.996700,1.952600,817.760010,-2948.699951,2.0054,1.682600,1.196800,74.624001,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.131800,27940.000000,862460.0,-0.002207,5.849100e+13,-897.840027,NaN,1.3561,3063.399902,...,1.013300e+11,357.269989,1476600.0,90.845001,1.306200e+09,2.373100,391.369995,2965.300049,NaN,0.494590,43.523998,138.520004,1.107900,0.919480,47.915001,NaN,1.510500e+12,3.4663,0.560950,4.130900,95531.000000,39.486000,-83.148003,0.084881,0.032222,0.001668,1.436500e+07,0.201020,-5.768800,1.2042,0.26290,8.13120,45119.000000,1.1764,3.218100e+14,3838.199951,0.40690,1,5,101.478569
3,3,0.152360,0.007259,780.099976,0.025179,0.519470,7.49140,112.510002,259490.0,7.781400e+13,-36.837002,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.840027,7.8490,1.037900,8.003000,12.349000,-195.279999,2.5598,92.141998,0.637890,1054.900024,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.989410,301200.000000,NaN,-0.000007,-9.299200e+13,-10.818000,1020.299988,2.9553,3342.500000,...,1.835400e+10,-3.429800,6485700.0,2120.000000,3.081200e+10,34.056000,157.429993,3724.500000,8.4211,0.407780,2971.199951,204.699997,-0.979980,9.940500,12011.000000,0.99898,5.063400e+13,1.2261,0.250200,0.729740,373690.000000,194.649994,120.930000,0.260710,0.234240,-0.002794,1.442300e+09,-0.011820,-34.858002,2.0694,0.79631,-16.33600,4952.399902,1.1784,4.533000e+12,4889.100098,0.51486,1,2,114.852005
4,4,0.116230,0.502900,-109.150002,0.297910,0.344900,-0.40932,2538.899902,65332.0,1.907200e+15,144.119995,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.900024,7.2783,3.975700,NaN,NaN,29520.000000,3.4225,96.724998,0.797250,215.570007,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.974130,142620.000000,231350.0,0.001258,1.012500e+16,51.507999,293.760010,1.3351,3042.100098,...,3.825200e+10,130.699997,102100.0,1951.800049,1.142800e+10,58.566002,176.830002,1279.000000,4.9662,0.479120,-70.278000,10.88700

In [ ]:
target = train_data.loc[:,'claim']

In [ ]:
target

0         1
1         0
2         1
3         1
4         1
         ..
957914    0
957915    1
957916    0
957917    1
957918    0
Name: claim, Length: 957919, dtype: int8

In [ ]:
train_data = train_data.drop(['id','claim'],axis=1)

In [ ]:
test_data.drop('id',axis=1,inplace=True)

In [ ]:
 train_data['std'] = train_data[features].std(axis=1)
test_data['std'] = test_data[features].std(axis=1)

train_data['min'] = train_data[features].min(axis=1)
test_data['min'] = test_data[features].min(axis=1)

train_data['max'] = train_data[features].max(axis =1)
test_data['max'] = test_data[features].max(axis=1)


In [ ]:
train_data = imputer.fit_transform(train_data)

In [ ]:
test_data = imputer.transform(test_data)

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
scaler = RobustScaler()

In [ ]:
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
features = features + ['N_missing','std_row','min','max','kurt']

In [ ]:
train_data = pd.DataFrame(train_data,columns =features)

In [ ]:
test_data = pd.DataFrame(test_data,columns =features)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
xtrain,xval, ytrain,yval= train_test_split(train_data, target, test_size= 0.3 ,random_state=1)

In [ ]:
! pip install catboost

     |████████████████████████████████| 67.4 MB 36 kB/s 


In [ ]:
!git clone --recursive https://github.com/microsoft/LightGBM


Cloning into 'LightGBM'...
remote: Enumerating objects: 23457, done.
remote: Counting objects: 100% (1134/1134), done.
remote: Compressing objects: 100% (602/602), done.
remote: Total 23457 (delta 709), reused 836 (delta 512), pack-reused 22323
Receiving objects: 100% (23457/23457), 18.13 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (17102/17102), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), done.        
remote: 

In [ ]:
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1 
!make -j$(nproc)


/content/LightGBM
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following B

In [ ]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [ ]:
!sudo python setup.py install --precompile

running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/dask.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest f

In [ ]:
%cd ../

/content


In [ ]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
LGBMClassifier().get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [ ]:
# paramsXGB = {'max_depth': 2, 'learning_rate': 0.021537077920105466, 'n_estimators': 3000, 'min_child_weight': 150, 'gamma': 0.11611920725914951, 'alpha': 0.0021839958087869794, 'lambda': 0.0018567979557499344, 'colsample_bytree': 0.7139742731494992, 'subsample': 0.6258627743440968,
#              'tree_method': 'gpu_hist',
#              'booster': 'gbtree',
#              'random_state': 228,
#              'use_label_encoder': False,
#              'objective': 'binary:logistic',
#              'eval_metric': 'auc',
#               }

In [ ]:
paramsXGB = {'max_depth': 3, 'learning_rate': 0.015702659398906191, 'n_estimators': 10000, 'min_child_weight': 25, 'gamma': 0.00010151247994797229, 'alpha': 7.148020356730985, 'lambda': 0.1378423649746119, 'colsample_bytree': 0.7969227570988136, 'subsample': 0.6382893449313995,
             'tree_method': 'gpu_hist',
             'booster': 'gbtree',
             'random_state': 228,
             'use_label_encoder': False,
             'eval_metric': 'auc'}

In [ ]:
# paramsCB = {'depth': 3, 'learning_rate': 0.014530866870832323, 'iterations': 6000, 'max_bin': 265, 'min_data_in_leaf': 14, 'l2_leaf_reg': 0.004427550682515904, 'subsample': 0.5402586792667279, 'grow_policy': 'SymmetricTree', 'leaf_estimation_method': 'Gradient',
#             'bootstrap_type': 'Bernoulli',
#             'random_seed': 228,
#             'loss_function': 'Logloss',
#             'eval_metric': 'AUC',}
#             'task_type': 'GPU' }

In [ ]:
paramsCB = {'depth': 3, 'learning_rate': 0.017585381726501453, 'iterations': 11636, 'max_bin': 461, 'min_data_in_leaf': 162, 'l2_leaf_reg': 0.02724781040038058, 'subsample': 0.6892384815879177, 'grow_policy': 'Depthwise', 'leaf_estimation_method': 'Gradient',
            'bootstrap_type': 'Bernoulli',
            'random_seed': 228,
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'task_type': 'GPU' }

In [ ]:
paramsLGBM = {'n_estimators':11990 , 'max_depth': 3, 'learning_rate': 0.016501612373246877, 'reg_alpha': 7.555087388180319, 'reg_lambda': 0.9534606245427513, 'num_leaves': 155, 'min_data_per_group': 177, 'min_child_samples': 150, 'colsample_bytree': 0.22781593823447946,
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'random_state': 228,
            'metric': 'auc',
            'device_type': 'gpu',}

In [ ]:
  models=[ 
            ######## First level ########
            [  LGBMClassifier(**paramsLGBM),
            #  XGBClassifier(**paramsXGB),
              
             CatBoostClassifier(**paramsCB),
            #  RandomForestClassifier (n_estimators=100, criterion="entropy", max_depth=5, max_features=0.5, random_state=1),
            #  ExtraTreesClassifier (n_estimators=100, criterion="entropy", max_depth=5, max_features=0.5, random_state=1,n_jobs=-1),
            #  GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, max_features=0.5, random_state=1),
             LogisticRegression(random_state=1,n_jobs=-1,max_iter=200),
             
             

             ],
            ######## Second level ########
            [LogisticRegression(random_state =1,max_iter=200
                                )]
            ]

In [ ]:
model=StackNetClassifier(models, metric="auc", folds=3,
	restacking=False,use_retraining=True, use_proba=True, 
	random_state=12345,n_jobs=1, verbose=1)

In [ ]:
model.fit(train_data,target)

Streaming output truncated to the last 5000 lines.
6654:	learn: 0.8267978	total: 10m 7s	remaining: 7m 34s
6655:	learn: 0.8267990	total: 10m 7s	remaining: 7m 34s
6656:	learn: 0.8267995	total: 10m 7s	remaining: 7m 34s
6657:	learn: 0.8268012	total: 10m 7s	remaining: 7m 34s
6658:	learn: 0.8268029	total: 10m 7s	remaining: 7m 34s
6659:	learn: 0.8268045	total: 10m 8s	remaining: 7m 34s
6660:	learn: 0.8268056	total: 10m 8s	remaining: 7m 34s
6661:	learn: 0.8268061	total: 10m 8s	remaining: 7m 34s
6662:	learn: 0.8268075	total: 10m 8s	remaining: 7m 34s
6663:	learn: 0.8268083	total: 10m 8s	remaining: 7m 33s
6664:	learn: 0.8268100	total: 10m 8s	remaining: 7m 33s
6665:	learn: 0.8268116	total: 10m 8s	remaining: 7m 33s
6666:	learn: 0.8268129	total: 10m 8s	remaining: 7m 33s
6667:	learn: 0.8268144	total: 10m 8s	remaining: 7m 33s
6668:	learn: 0.8268161	total: 10m 8s	remaining: 7m 33s
6669:	learn: 0.8268179	total: 10m 9s	remaining: 7m 33s
6670:	learn: 0.8268185	total: 10m 9s	remaining: 7m 33s
6671:	learn: 0

In [ ]:
preds=model.predict_proba(test_data)

====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 


In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle-sep_tabular_competition/data/sample_solution.csv')

In [ ]:
submission.claim = preds[:,1]

In [ ]:
submission

,id,claim
0,957919,0.571806
1,957920,0.128191
2,957921,0.627401
3,957922,0.127653
4,957923,0.139785
...,...,...
493469,1451388,0.820465
493470,1451389,0.124176
493471,1451390,0.759032
493472,1451391,0.136795


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle-sep_tabular_competition/data/submission12.csv',index=False)